# Sparse tensors

A new functionality of Tensor Fox is the support for sparse tensors. Let $T \in \mathbb{R}^{I_1 \times \ldots \times I_L}$ be a sparse tensor with *nnz* nonzero entries. This tensor is represented as a triple *[data, idxs, dims]*, where *data* is an array of size $nnz$ such that *data[i]* is the $i$-th nonzero entry of $T$, with corresponding index *idxs[i]*. It is necessary to pass *idxs* also as an array of shape $nnz \times L$. Finally, we have that *dims* $= [I_1, \ldots, I_L]$ are the dimwnsions of the tensor. 

In [1]:
import numpy as np
import TensorFox as tfx
import matplotlib.pyplot as plt

## Small example

Below we start with a fourth order sparse tensor $10 \times 10 \times 10 \times 10$ with rank $R = 5$ such that each factor matrix has 5 nonzero entries. We can generate sparse factor matrices with the function *gen_rand_sparse_tensor* with inputs **dims**, **R**, **nnz** as shown below. We remark that the sparsity of the tensor is not necessarily equal to the sparsity of its factor matrices.

In [2]:
# Dimensions of the tensor.
dims = [10, 10, 10, 10]

# Rank of the tensor.
R = 5

# Number of nonzero entries of each factor matrix. 
nnz = 10

# Generate sparse tensor.
factor_dims_prod = tfx.multiply_dims([dims[0], R])
print('Factor matrix sparsity level =', round(100 * (factor_dims_prod - nnz)/factor_dims_prod, 2), '%')
data, idxs, dims, factors = tfx.gen_rand_sparse_tensor(dims, R, nnz)
tensor_dims_prod = tfx.multiply_dims(dims)
print('Tensor sparsity level =', round(100 * (tensor_dims_prod - len(data))/tensor_dims_prod, 2), '%')

# Define sparse tensor.
T = [data, idxs, dims]

Factor matrix sparsity level = 80.0 %
Tensor sparsity level = 87.4 %


We remark that it is not possible to use $\verb|tol| \_ \verb|mlsvd| = -1$ for sparse tensors. In the case the user sets $\verb|display| = 3$ or $4$, the program computes only the error associated with the nonzero entries of $T$, otherwise we would face memory issues. 

In [3]:
class options:
    display = 1
    
factors, output = tfx.cpd(T, R, options)

Sparse tensor detected
    nnz = 1260
    Sparsity level = 87.4 %
-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4
    Compression detected
    Compressing from (10, 10, 10, 10) to (5, 5, 5, 5)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 85
    Relative error = 1.2244622813077945e-14
    Accuracy =  100.0 %


The relative error above only take in account the nonzero entries of $T$ (the same is true for the compression error when displayed). The factor matrices of the decomposition may introduce small errors when approximating the zeros, and these small errors summed together does increase the actual error of the CPD. Thus the relative error above is a lower bound to the actual error, but usually close enough.

We can convert the factor matrices to tensor sparse format just like the original input. For that we use the function **cpd2sparsetens**. The variables **idxs_sp** and **dims_sp** are the same of **idxs** and **dims**, we just repeated them to be concise with the format. 

In [4]:
# Generate the sparse format from the factors.
data_approx, idxs_sp, dims_sp = tfx.cpd2sparsetens(factors, idxs, dims)

# Compute the error.
np.linalg.norm(data - data_approx)/np.linalg.norm(data)

1.2244622813077945e-14

The function **cpd2sparsetens** is flexible enough so we can even check for indexes outside **idxs**, that is, the values of the tensor we know that should be zero. This is a sanit check that can be helpful sometimes.

In [5]:
# Generate a few indexes outside idxs.
idxs_rnd = np.random.randint(0, 10, size=[1000, len(dims)])
idxs_rnd = [idx for idx in idxs_rnd if idx not in idxs]
print('Generated', len(idxs_rnd), 'indexes')

Generated 25 indexes


In [6]:
# Show the values of the approximated tensor for these indexes. They should be close to zero. 
data_rnd_approx, idxs_sp, dims_sp = tfx.cpd2sparsetens(factors, idxs_rnd, dims)
data_rnd_approx

array([-2.31813972e-67,  0.00000000e+00,  0.00000000e+00,  2.67666419e-66,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  7.69648378e-66,
        0.00000000e+00,  2.71622246e-67,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  1.11853999e-66,  8.28962224e-79,
        0.00000000e+00])

Since this is a small example we can put everything in dense format and verify what is the actual error (this won't be possible for really large tensors). Regardless, the main point of the CPD is to approximate the nonzero entries, which is always accomplished.

In [7]:
# Generate the coordinate (dense) format of the approximation.
T_approx = tfx.cpd2tens(factors)

# Generate the dense format from the sparse representation.
T_dense = tfx.sparse2dense(data, idxs, dims)

# Compute the error.
np.linalg.norm(T_dense - T_approx)/np.linalg.norm(T_dense)

1.224164023490288e-14

## Memory cost

There is a big reduction in memory cost when working with sparse representations. Below we show a graph with the maximum memory cost attained in the computation of the CPDs of sparse $n \times n \times n$ tensors (blue curve) vs. the cost to store these tensors in dense format. As we can see, the difference is substantial, Tensor Fox does avoid the intermediate memory explosion problem. For instance, the sparse approach requires $3648$ megabytes when $n = 30000$, whereas the dense approach requires approximately $205$ terabytes. 

![sparse](sparse.png)

## Big example

The amazing memory reduction showed above is due to the fact that Tensor Fox needs $\mathcal{0}(I_\ell^2)$ flops of memory for each unfolding whereas the other classic methods needs $\mathcal{O}(\prod_{k \neq \ell} I_k)$ flops. Still, one can face tensor with dimensions at the order of billions or more, and in these cases even Tensor Fox will blow up the memory. To address this issue one can "divide" the dimensions in smaller pieces and then compute a CPD in a space with more dimensions, but lower ones. This approach is computationally feasible and can find good solutions. For more detail about this approach see (). 

In the example we generate a fourth order tensor $10^3 \times 10^3 \times 10^3 \times 10^3$ with rank $R = 5$. 

In [8]:
# Dimensions of the tensor.
dims = [10**3, 10**3, 10**3, 10**3]

# Rank of the tensor.
R = 5

# Number of nonzero entries of each factor matrix. 
nnz = 10

# Generate sparse tensor.
factor_dims_prod = tfx.multiply_dims([dims[0], R])
print('Factor matrix sparsity level =', round(100 * (factor_dims_prod - nnz)/factor_dims_prod, 2), '%')
data, idxs, dims, factors = tfx.gen_rand_sparse_tensor(dims, R, nnz)
tensor_dims_prod = tfx.multiply_dims(dims)
print('Tensor sparsity level =', round(100 * (tensor_dims_prod - len(data))/tensor_dims_prod, 7), '%')

# Define sparse tensor.
T = [data, idxs, dims]

Factor matrix sparsity level = 99.8 %
Tensor sparsity level = 99.999999 %


In [9]:
options.display = 1
factors, output = tfx.cpd(T, R, options)

Sparse tensor detected
    nnz = 10000
    Sparsity level = 99.999999 %
-----------------------------------------------------------------------------------------------
Computing MLSVD
    Compressing unfolding mode 1
    Compressing unfolding mode 2
    Compressing unfolding mode 3
    Compressing unfolding mode 4


->  mkl_sparse_d_spmmd returned 5 (SPARSE_STATUS_INTERNAL_ERROR). Using standard scipy dot.


    Compression detected
    Compressing from (1000, 1000, 1000, 1000) to (5, 5, 5, 5)
-----------------------------------------------------------------------------------------------
Type of initialization: random
-----------------------------------------------------------------------------------------------
Computing CPD
Final results
    Number of steps = 64
    Relative error = 4.6285761822170106e-14
    Accuracy =  100.0 %


In this particular example note that the mode 4 unfolding raised a warning. The program has three routines to perform sparse matrix multiplication, in order: the package *sparse_dot_mkl*, *Scipy* and Tensor Fox's routine itself. This order relies solely on speed. When the first one failed, the program used the sparse matrix multiplication from Scipy. The sparse multiplication from Tensor Fox usually will be called only when the tensor is so large that it can't be handled by none of the previous routines. With that regard Tensor Fox sparse multiplication can handle sparse tensors as long as the number on nonzero entries fits in memory.

We finish with a few observations about the parameter $\verb|mkl| \_ \verb|dot|$. By default the program uses a specific MKL sparse dot function to perform matrix-matrix multiplication. If the package *sparse_dot_mkl* is not installed, then the program uses the sparse dot product from Scipy. You can set $\verb|mkl| \_ \verb|dot| = False$ to avoid the package *sparse_dot_mkl* completely.